<a href="https://colab.research.google.com/github/noelzogbladan/K_flix.py/blob/master/Scrapping_donn%C3%A9es_modmob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
pip install contextily

In [ ]:
% pip install beautifulsoup4
% pip install requests

In [ ]:

from urllib.request import urlopen
from bs4 import BeautifulSoup, SoupStrainer

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium

import rtree

from shapely.geometry import Point
from shapely.ops import transform

from IPython.display import display

In [ ]:
import contextily as ctx
import requests
import tempfile
import zipfile


#Communes de France

In [ ]:

url = 'https://www.data.gouv.fr/fr/datasets/r/17062524-991f-4e13-9bf0-b410cc2216fd'
temporary_location = tempfile.gettempdir()

def download_unzip(url, dirname = tempfile.gettempdir(), destname = "borders"):
  myfile = requests.get(url)
  open(dirname + '/' + destname + '.zip', 'wb').write(myfile.content)
  with zipfile.ZipFile(dirname + '/' + destname + '.zip', 'r') as zip_ref:
      zip_ref.extractall(dirname + '/' + destname)

In [ ]:
download_unzip(url)
communes = gpd.read_file(temporary_location + "/borders/communes-20210101.shp")
communes.head()

In [ ]:
communes.shape

In [ ]:
haute_gar = communes[communes.insee.str.startswith("31")]
ax = haute_gar.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, crs = haute_gar.crs.to_string())
ax

#Données metro, tram et RER ile de france

In [ ]:
url="https://www.data.gouv.fr/fr/datasets/r/95bb11ed-b606-437e-a5b6-72742678cb3e"

In [ ]:
transport_idf = gpd.read_file(url)
transport_idf.head()

In [ ]:
transport_idf.shape

#Arrêts transport en commun à Lyon

In [ ]:
url_lyon="https://www.data.gouv.fr/fr/datasets/r/3c4701aa-6ac8-467d-be11-2a9808988097"

In [ ]:
transport_lyon = gpd.read_file(url_lyon)
transport_lyon.head()

In [ ]:
transport_lyon.shape

#Transport en commun toulouse

In [ ]:
url_toulouse_bus="https://www.data.gouv.fr/fr/datasets/r/c6787c06-fc3c-49b1-b9ca-9ae542f788e0"

In [ ]:
transport_toulouse_bus = gpd.read_file(url_toulouse_bus)
transport_toulouse_bus.head()

#Scraping des données bus

In [ ]:
page= requests.get("https://transport.data.gouv.fr/datasets?q=arrets+bus")
soup= BeautifulSoup(page.content,"html.parser")

In [ ]:
print(soup)

In [ ]:
week= soup.find(class_="dataset-page")
#items= week.find_all(class_='tombstone-container')


In [ ]:
#http = httplib2.Http()
#status, response = http.request('https://transport.data.gouv.fr/datasets?q=arrets+bus')
#for link in BeautifulSoup(response, 'html.parser',
 #                            parseOnlyThese=SoupStrainer('a')):
 #   if link.has_attr('href'):
 #       lien= link['href']
        #page1= requests.get("http://lien")
        #soup1= BeautifulSoup(page.content,"html.parser")
        


In [ ]:
url = urlopen("https://transport.data.gouv.fr/datasets?q=arrets+bus").read()
soup = BeautifulSoup(url)
for line in soup.find_all('a'):
    lien=line.get('href')
    print(lien)
    

#Fonctions sur les liens sur une page

In [ ]:
import urllib
import lxml.html
from urllib.parse import urlparse, parse_qsl, urljoin

In [ ]:
def get_dom(url):
    connection = urlopen(url)
    return lxml.html.fromstring(connection.read())

def get_links(url):
    return resolve_links((link for link in get_dom(url).xpath('//a/@href')))

def guess_root(links):
    for link in links:
        if link.startswith('http'):
            parsed_link = urlparse(link)
            scheme = parsed_link.scheme + '://'
            netloc = parsed_link.netloc
            return scheme + netloc
def resolve_links(links):
    root = guess_root(links)
    for link in links:
        if not link.startswith('http'):
            link = urljoin(root, link)
        yield link  

In [ ]:
for link in get_links('https://www.data.gouv.fr/fr/datasets/'):
  print(link)

In [ ]:
url="https://transport.data.gouv.fr/datasets?q=arrets+bus"
page=urlopen(url)
data=page.read().decode().split("</a>")
tag="<a href=\""
endtag="\">"
for item in data:
    if "<a href" in item:
        try:
            ind = item.index(tag)
            item=item[ind+len(tag):]
            end=item.index(endtag)
        except: pass
        else:
            [print (item[:end])

In [ ]:
#!/usr/bin/env python
import sys
import lxml.html
import fnmatch
from urllib.request import urlopen as open

try:
    import urltools as urltools
except ImportError:
    sys.stderr.write('To normalize URLs run: `pip install urltools --user`')
    urltools = None


def get_host(url):
    p = urlparse(url)
    return "{}://{}".format(p.scheme, p.netloc)


if __name__ == '__main__':
    url = sys.argv[1]
    host = get_host(url)
    glob_patt = len(sys.argv) > 2 and sys.argv[2] or '*'

    doc = lxml.html.fromstring(urlopen(url).read())
    links = doc.xpath('//a[@href]')

    for link in links:
        href = link.attrib['href']

        if fnmatch.fnmatch(href, glob_patt):

            if not href.startswith(('http://', 'https://' 'ftp://')):

                if href.startswith('/'):
                    href = host + href
                else:
                    parent_url = url.rsplit('/', 1)[0]
                    href = urlparse.urljoin(parent_url, href)

                    if urltools:
                        href = urltools.normalize(href)

            print (href)